<a href="https://colab.research.google.com/github/varaha-Maithreya/Person-Reidentification/blob/main/videoTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/varaha-Maithreya/Person-Reidentification.git

In [ ]:
pip install timm scipy einops yacs opencv-python tensorboard pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!ls 

!pwd

config	 datasets  loss   processor  solver  train.py  Vis.ipynb
configs  LICENSE   model  README.md  thop    utils     vis.py
/content/Person-Reidentification


In [ ]:
cd Person-Reidentification/


/content/Person-Reidentification


In [ ]:
import os
import torch, cv2
import numpy as np
from config import cfg
import argparse
from datasets import make_dataloader
from model import make_model
from processor import do_inference
from utils.logger import setup_logger
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from model.make_model import __num_of_layers
from model.make_model import build_transformer_local
from model.backbones.vit_pytorch import vit_base_patch16_224_PiT, vit_small_patch16_224_PiT, deit_small_patch16_224_PiT

In [ ]:
cfg.merge_from_file("/content/Person-Reidentification/configs/iLIDS-VID/pit.yml")

cfg.merge_from_list(['TEST.VIS', "True"])
cfg.freeze()

In [ ]:
output_dir = cfg.OUTPUT_DIR
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
torch.cuda.is_available()
torch.cuda.device_count()

1

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = cfg.MODEL.DEVICE_ID

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] 

'0'

In [ ]:
epochs = cfg.SOLVER.MAX_EPOCHS
eval_period = cfg.SOLVER.EVAL_PERIOD
OUTPUT_DIR = cfg.TEST.WEIGHT


In [ ]:
val_transforms = T.Compose([
   T.Resize(cfg.INPUT.SIZE_TEST, interpolation=3),
   T.ToTensor(),
   T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [ ]:
__factory_T_type = {
    'vit_base_patch16_224_PiT': vit_base_patch16_224_PiT,
    'deit_base_patch16_224_PiT': vit_base_patch16_224_PiT,
    'vit_small_patch16_224_PiT': vit_small_patch16_224_PiT,
    'deit_small_patch16_224_PiT': deit_small_patch16_224_PiT
}

__num_of_layers = {
    '1x210' : 1,    # global
    '2x105' : 2,    # horizontal
    '3x70'  : 3,
    '5x42'  : 5,
    '6x35'  : 6,
    '7x10'  : 7,
    '105x2' : 2,    # vertical
    '70x3'  : 3,
    '42x5'  : 5,
    '35x6'  : 6,
    '10x7'  : 7,
    '6p'    : 6,    # patch
    '14p'   : 14,
    '15p'   : 15,
    'NULL'  : 0,
}

In [ ]:
train_loader, val_loader, num_query, num_classes, camera_num, view_num = make_dataloader(cfg)



This dataset has been downloaded.
Creating splits
Totally 10 splits are created, following Wang et al. ECCV'14
Split file is saved to ../../data/i-LIDS-VID/splits.json
Splits created
# train identites: 150, # test identites 150
=> iLIDS-VID loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   150 |      300
  query    |   150 |      150
  gallery  |   150 |      150
  ------------------------------
  total    |   300 |      600
  number of images per tracklet: 22 ~ 192, average 70.8
  ------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pwd
num_trails = 10

model = make_model(cfg, num_class=num_classes, camera_num=camera_num, view_num = view_num)

param_dict = torch.load('/content/jx_vit_base_p16_224-80ecf9dd.pth')
model.load_state_dict(param_dict, strict=False)




model.to("cuda")
model.eval()



/content/Person-Reidentification
using Transformer_type: vit_base_patch16_224_PiT as a backbone
using stride: [12, 12], and patch number is num_y21 * num_x10
camera number is : 2
using SIE_Lambda is : 3.0
using drop_out rate is : 0.0
using attn_drop_out rate is : 0.0
using drop_path rate is : 0.1
Resized position embedding from size:torch.Size([1, 197, 768]) to size: torch.Size([1, 211, 768]) with height:21 width: 10
Loading pretrained ImageNet model......from /content/jx_vit_base_p16_224-80ecf9dd.pth
using shuffle_groups size:2
using shift_num size:5
using divide_length size:5
===========building transformer with JPM module ===========


build_transformer_local(
  (base): PiT(
    (patch_embed): PatchEmbed_overlap(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(12, 12))
    )
    (temporal_pos_drop): Dropout(p=0.0, inplace=False)
    (spatial_blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Bl

In [ ]:
score, feat, diversity = model(img, target, cam_label=target_cam, view_label=target_view )

In [ ]:
from torchvision.io import read_image
img = read_image('/content/pexels-lukas-rychvalsky-670786.jpg')

In [ ]:
img.shape

In [ ]:
from PIL import Image
directory = '/content/pexels-lukas-rychvalsky-670786.jpg'
img = Image.open(directory)

In [ ]:
val_transforms(img)

In [ ]:
img = torch.stack([val_transforms(img)], 0).unsqueeze(0).to("cuda")

In [ ]:
attns = model(img, cam_label=[0])

In [ ]:
 attn_base, attn_head = attns
 att_base, att_head = torch.stack(attn_base).squeeze(1), torch.stack(attn_head).squeeze(1)


In [ ]:
 # Average the attention weights across all heads.
att_base, att_head = torch.mean(att_base, dim=1), torch.mean(att_head, dim=1)


In [ ]:
# To account for residual connections, we add an identity matrix to the
    # attention matrix and re-normalize the weights.
residual_att_base, redisual_att_head = \
    torch.eye(att_base.size(1)).to("cuda"), \
    torch.eye(att_head.size(1)).to("cuda")
aug_att_base, aug_att_head = \
    att_base + residual_att_base, \
    att_head + redisual_att_head
aug_att_base, aug_att_head = \
    aug_att_base / aug_att_base.sum(dim=-1).unsqueeze(-1), \
    aug_att_head / aug_att_head.sum(dim=-1).unsqueeze(-1)


In [ ]:
 # Recursively multiply the weight matrices
joint_attentions = torch.zeros(aug_att_base.size()).to("cuda")
joint_attentions[0] = aug_att_base[0]

for n in range(1, aug_att_base.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_base[n], joint_attentions[n - 1])


In [ ]:
im = Image.open(open('/content/pexels-lukas-rychvalsky-670786.jpg', 'rb'))

In [ ]:
# Attention from the att_base output token to the input space.
v = joint_attentions[-1]
img_H, img_W = 21, 10
mask = v[0, 1:].reshape(img_H, img_W).detach().cpu().numpy()
mask = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
result = (mask * im).astype("uint8")


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
plt.figure()
plt.imshow(result)
plt.show()

In [ ]:
plt.figure()
plt.imshow(mask[:,:,0])
plt.show()

